# Big Data Analytics

## add names here:




### The aim of this project is to use the IMDB Dataset: (Link) and use data such as actors, director, genre, isAdult and runtime to predict how well a movie will perform. We use the boxoffice and userrating as measures of performance. We will train our model with 80% of the data and test it on the remaining 20%. We will also give our predicitons for upcoming movies that do not have any ratings/boxoffice yet.

### Our hypothesis is that actors & director will positively influence the performance while the variable runtime will negatively influence it. 



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.linear_model import Ridge,LinearRegression,Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline



# Load Data

## title basics 

In [5]:
print("loading data")
df = pd.read_csv("data/title.basics.tsv",sep="\t")
print(df.shape)
print("Dropping TV-series")
df = df[(df['endYear']=='\\N')]
print("dropping endYear column")
df.drop(["endYear"],axis=1,inplace=True)
# specific TV episodes we want to drop
print("Selecting movies")
df = df[(df['titleType']=="movie")]
#replace \N with NAN
print("replacing \\N")
df.replace("\\N",np.nan,inplace=True)
#drop NAN
print("dropping NAN's")
df.dropna(inplace=True)
#converting startYear to int
print("converting startYear, isAdult and runtimeMinutes to int")
df['startYear'] = df['startYear'].astype(int)
df['isAdult'] = df['isAdult'].astype(int)
df['runtimeMinutes'] = df['runtimeMinutes'].astype(int)

#select correct time frame
print("selecting correct time frame")
df = df[(df['startYear']>=2000) & (df['startYear']<=2020)]

df.reset_index(inplace=True,drop=True)
df.to_csv("data/title.basics_preprocessed.csv",index=False)
print(df.shape)
df.head()

loading data
/home/flo/anaconda3/envs/datascience/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
(7869186, 9)
Dropping TV-series
dropping endYear column
Selecting movies
replacing \N
dropping NAN's
converting startYear, isAdult and runtimeMinutes to int
selecting correct time frame
(189861, 8)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,67,Drama
1,tt0016906,movie,Frivolinas,Frivolinas,0,2014,80,"Comedy,Musical"
2,tt0019996,movie,Hongxia,Hongxia,0,2011,94,Action
3,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
4,tt0036177,movie,Muhomatsu no issho,Muhomatsu no issho,0,2008,100,"Action,Adventure"


In [6]:
df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,67,Drama
1,tt0016906,movie,Frivolinas,Frivolinas,0,2014,80,"Comedy,Musical"
2,tt0019996,movie,Hongxia,Hongxia,0,2011,94,Action
3,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
4,tt0036177,movie,Muhomatsu no issho,Muhomatsu no issho,0,2008,100,"Action,Adventure"
...,...,...,...,...,...,...,...,...
189856,tt9916362,movie,Coven,Akelarre,0,2020,92,"Adventure,Drama,History"
189857,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,123,Drama
189858,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,Documentary
189859,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary


In [ ]:




df = pd.read_csv("filename")


# extract train and test variables
data = df.to_numpy()
X = df.loc[:, df.columns != ''].to_numpy()   # enter column name of test variable
y = np.log(df['']).to_numpy()                # enter column name of test variable

print(X.shape)
print(y.shape)

# Data Preprocessing

## Steps performed

### dropping columns, imputing missing values, categorical datapoints


# Kfold to decrease potential overfitting

In [3]:
from sklearn.model_selection import KFold

def kfold_validation(X,y,model):
    start = time.time()
    kf = KFold(n_splits=5,shuffle=True, random_state=69420) 
    mses = []
    models = []
    count = 0
    for trainIndices,testIndices in kf.split(X):
        print(f"iteration: {count}")
        Xtrain,Xval = X[trainIndices,:],X[testIndices,:]
        ytrain,yval = y[trainIndices],y[testIndices]
        model.fit(Xtrain,ytrain)
        yhat = model.predict(Xval)
        mse = np.sum(np.square(yval-yhat))/yval.size # mean squared error
        mses.append(mse)
        models.append(model)
        count+=1
    print(f"time used (seconds): {time.time()-start}")
    return mses, models


# Standardise to make data more comparable

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Xtrain,Xtest, ytrain,ytest = train_test_split(X,y,shuffle=True,train_size=0.8)

def standardise_data(X):
    mean = np.mean(X)
    std = np.std(X)
    X_std = (X-mean)/std
    return X_std, mean, std

#kf = KFold(n_splits=1,shuffle=True, random_state=69420) 

Xtrain_std, X_train_mean, X_train_std_div = standardise_data(Xtrain)
Xtest_std = (Xtest-X_train_mean)/X_train_std_div


NameError: name 'X' is not defined

# Linear Regression

In [5]:
model = LinearRegression()
mses,models = kfold_validation(Xtrain,ytrain,model)
yhat=models[np.argmin(mses)].predict(Xtest)
np.mean(np.square(yhat-ytest))

NameError: name 'Xtrain' is not defined

# Random Forest

In [6]:
from sklearn.ensemble import RandomForestRegressor 

model = RandomForestRegressor()
mses,models = kfold_validation(Xtrain,ytrain,model)
yhat=models[np.argmin(mses)].predict(Xtest)
np.mean(np.square(yhat-ytest))


NameError: name 'Xtrain' is not defined

# Data Visualisation